In [9]:
from langgraph.graph import StateGraph
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel

# Define input and output schema
class InputState(BaseModel):
    name: str

class OutputState(BaseModel):
    message: str
    description: str 

# Create the graph with schema
graph = StateGraph(input=InputState, output=OutputState)

# Fix: Use attribute access
graph.add_node("greet", RunnableLambda(lambda state: {"message": f"Hello, {state.name}!"}))

graph.set_entry_point("greet")
app = graph.compile()

# Run the graph
result = app.invoke({"name": "Satish"})
print(result)  # Output: {'message': 'Hello, Satish!'}

{'message': 'Hello, Satish!'}


C:\Users\GUR56937\AppData\Local\Temp\ipykernel_21600\834860050.py:14: LangGraphDeprecationWarning: Initializing StateGraph without state_schema is deprecated. Please pass in an explicit state_schema instead of just an input and output schema.
  graph = StateGraph(input=InputState, output=OutputState)


In [20]:
from langgraph.graph import StateGraph
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel

# Define the shared state schema
class State(BaseModel):
    input: str
    text: str = ""
    response: str = ""

# Step 1: Extract input
step1 = RunnableLambda(lambda state: {"text": state.input})

# Step 2: Transform text to uppercase
step2 = RunnableLambda(lambda state: {"text": state.text.upper()})

# Step 3: Create a response
step3 = RunnableLambda(lambda state: {"response": f"Processed: {state.text}"})

# Create the graph
graph = StateGraph(state_schema=State)

# Add nodes
graph.add_node("step1", step1)
graph.add_node("step2", step2)
graph.add_node("step3", step3)

# Define edges
graph.add_edge("step1", "step2")
graph.add_edge("step2", "step3")

# Set entry and finish points
graph.set_entry_point("step1")
graph.set_finish_point("step3")

# Compile the graph
app = graph.compile()

# Run the graph
result = app.invoke({"input": "hello langgraph"})
print(result)  # Output: {'input': 'hello langgraph', 'text': 'HELLO LANGGRAPH', 'response': 'Processed: HELLO LANGGRAPH'}


{'input': 'hello langgraph', 'text': 'HELLO LANGGRAPH', 'response': 'Processed: HELLO LANGGRAPH'}


In [21]:
from typing import List

from langchain_core.runnables import RunnableLambda
from typing_extensions import TypedDict


class Args(TypedDict):
    a: int
    b: List[int]


def f(x: Args) -> str:
    return str(x["a"] * max(x["b"]))


runnable = RunnableLambda(f)
as_tool = runnable.as_tool(
    name="My tool",
    description="Explanation of when to use tool.",
)

print(as_tool)  # Output: 'My tool'

as_tool.invoke({"a": 2, "b": [1, 2, 3]})  # Output: '6'


print(as_tool.description)

as_tool.args_schema.model_json_schema()

name='My tool' description='Explanation of when to use tool.' args_schema=<class 'langchain_core.tools.convert.My tool'> func=<function convert_runnable_to_tool.<locals>.invoke_wrapper at 0x00000131A992ECA0> coroutine=<function convert_runnable_to_tool.<locals>.ainvoke_wrapper at 0x00000131A992E200>
Explanation of when to use tool.


{'properties': {'a': {'title': 'A', 'type': 'integer'},
  'b': {'items': {'type': 'integer'}, 'title': 'B', 'type': 'array'}},
 'required': ['a', 'b'],
 'title': 'My tool',
 'type': 'object'}

In [8]:
from typing import Any, Dict


def g(x: Dict[str, Any]) -> str:
    return str(x["a"] * max(x["b"]))


runnable = RunnableLambda(g)
as_tool = runnable.as_tool(
    name="My tool",
    description="Explanation of when to use tool.",
    arg_types={"a": int, "b": List[int]},
)

as_tool.invoke({"a": 3, "b": [1, 2,5]})

'15'

In [11]:
from pydantic import BaseModel, Field


class GSchema(BaseModel):
    """Apply a function to an integer and list of integers."""

    a: int = Field(..., description="Integer")
    b: List[int] = Field(..., description="List of ints")


runnable = RunnableLambda(g)
as_tool = runnable.as_tool(GSchema)


as_tool.invoke({"a": 3, "b": [1, 2,5,16]})

'48'

In [18]:
from langchain_core.documents import Document
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
    ),
]

vectorstore = InMemoryVectorStore.from_documents(
    documents, embedding=OpenAIEmbeddings(),
    verify=False
)

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

APIConnectionError: Connection error.